## 4.3 – Working With Files
### Files
At some point once you start writing large applications you are likely to want to read from or write to a file on your system. You might be loading in a bunch of data to process, or you might be saving the progress that the player has made in your game. Even simple programs might want to store and load configuration settings in a separate file.

To open a file in Python, we need to create a *stream*. A stream is an object which allows us to read and write to a sequence of data. That sequence of data does not need to be a file: we might have reason to create an in-memory stream, or we might be reading from a network connection (e.g. over the Internet). But for the purposes of this section, since we are dealing with files, the streams we are using are *file objects*.

To open a file object, we can use the builtin `open` function. This function has many features, but most of the time you will only need to supply the first two arguments:
```python
open(filename, mode)
```

The first argument, `filename`, is the path of the file you wish to open. Normally you will provide the path as a string, though other options exist. If you write the name of a file on its own, Python will look in the same folder as the script that is running. So we can open the previous notebook as a file by using `"4-2-random.ipynb"`. (Note: if you try this, bear in mind that Jupyter files are stored in a [special format](https://nbformat.readthedocs.io/en/latest/) that you will need to process to be able to do anything sensible. There is an [inbuilt module](https://docs.python.org/3/library/json.html) that can help if you're up for it!)

You can use normal Unix path commands (such as `..` to move up one directory) as well, so another valid path might be `"../Week 1/1-1-expressions.ipynb"`.

The second argument, `mode`, is a string which controls two things. First, do we want to open the file in *text mode* or in *binary mode*? We will only use text mode in this section, but you might want binary mode if you are dealing with non-text files. The second option is whether we want to read from or write to the file.

There are a few other options for appending, creating, or updating (both read and write) as well, but most of the time, these are the only modes you will need:
* `"r"` read in text mode (same as `"rt"`)
* `"w"` write in text mode (same as `"wt"`)
* `"rb"` read in binary mode
* `"wb"` write in binary mode

So the following line will open the previous notebook file to read in text mode, and store the file object (the stream) in a variable called `file`:

In [1]:
file = open("4-2-random.ipynb", "r")

Note that the file object itself is just a “connection” to the file. The file has been opened but its contents have not been read yet. 

To read or write, we need to use the corresponding methods.

To read from a file:
* `file.read()` which will read the entire file and return the result as a string
* `file.read(n)` which reads `n` characters and returns the result as a string
* `file.readline()` which reads until the end of a line and returns the result as a string
* `file.readlines()` which reads the entire file and returns a *list of strings* broken into lines

To write to a file:
* `file.write(text)` which will write `text` to the file
* `file.writelines(lines)` which will write a list of lines `lines` to a file

##### Side Note: Lines

While we commonly think about text files being split into lines, it is worth pausing for a second to think about what this actually means. A newline in a text file is actually represented by one or more special characters in the stream of text. It is only when we display it on the screen that we choose to split the text at this point: the actual text data is all sequential. 

There is some confusion between platforms for historical reasons, but *for the most part* modern applications will understand a single character called the *linefeed* to represent a newline. This character has the decimal ASCII value of 10. But we already use linefeeds (newlines) to even structure our code, there's one at the end of each line! We often want to be able to encode a newline without actually breaking the line of code, so there is a special *escape sequence* which is extremely common: `\n`.

In [2]:
print("This string\nis actually\nprinted over\nmultiple lines!")

This string
is actually
printed over
multiple lines!


It is also possible in Python to use a triple-quoted string `"""..."""`, which will capture any newlines and insert them into the string, unless there is a backslash at the end of the line:

In [3]:
my_string = """this string
is actually
printed \
over
multiple lines!"""
print(my_string)

this string
is actually
printed over
multiple lines!


When we read lines from a text file (using `.readlines()`), the newline character is included at the end of each string in the list. There is a demonstration in the code below.

The backslash is used for a variety of escape characters in strings, so if you want an actual backslash you need to “escape it” using another backslash! So `\\` will insert a single `\`. Notice in the code below we had to replace the single character `\n` with `\\n` so that it would display as `\n` instead of rendering as an actual line break, but the lines still display across multiple lines (despite no longer containing a linefeed character) because the `print` function adds a newline at the end of each line when it displays them. (Keep up with that...? Guess what will happen if you delete the `.replace` method in the code below, then try it and rerun the cell.)

In [4]:
file = open("./resources/text_file.txt", "r")

lines = file.readlines()

for i in range(0, len(lines)):
    print(lines[i].replace("\n", "\\n"))

This is line 1 of a simple text file\n
This is line 2\n
\n
That previous line was blank\n
This is the final line, it does not end in a line break


### Closing Files
The code above is slightly lazy. When Python opens a file, it does so with the help of the operating system, which will only allow Python to open a certain number of files. If we keep opening files we will eventually hit the limit. Strictly speaking, we should close them after we are done.

And we can do this with `file.close()`. This mantra: “always close your files!” has been the go-to advice in programming tutorials and university units for decades.

***But*** *let's immediately caveat that advice.* Programming languages have evolved since the original mechanisms were invented, and now offer tools which can do this work for you. You never actually need to call the `.close()` method of a file object, if you open it correctly in the first place.

The correct solution in modern Python is to use the `with` keyword. This is a block of code which allows us to open some resource, then when the block is exited it will properly perform any clean up the object requires, such as closing a file.

In [5]:
with open("./resources/text_file.txt", "r") as file:
    lines = file.readlines()

for i in range(0, len(lines)):
    print(lines[i].replace("\n", "\\n"))

This is line 1 of a simple text file\n
This is line 2\n
\n
That previous line was blank\n
This is the final line, it does not end in a line break


This approach is better than manually opening and closing the file, partly because it means you don't have to remember to close it manually after opening, but also because it will still try to close the file even if something odd happens, like if an error occurs while reading the file.

Notice that in the code above the file is only open for a single line: long enough for its entire contents to be read into the variable called `lines`. We then exit the `with` block, which closes the file, and continue processing the text however we like (in this case, just printing it).

You should always use a `with` block when dealing with files. If you forget (or are feeling lazy), the file will be still be closed when your application closes. So if you are writing short programs that process text and exit, you are probably okay. If your programs run continually, you need to make sure they handle file opening and closing properly.

### Writing
Writing to files should be pretty self explanatory. Just beware that you won't necessarily get the same sort of precautions you might be used to with normal applications. When you open a file in write mode, any text written will **overwrite** the contents of that file, without warning.

In [6]:
with open("test_file.txt", "w") as file:
    file.write("Imagine there is a really long important piece of information here")
    
with open("test_file.txt", "w") as file:
    file.write("whoops")
    
with open("test_file.txt", "r") as file:
    print(file.read())

whoops


Rather than having lots of calls to `.write` in loops or in different places in your code, a good pattern to follow is to build up the data you wish to write to the file first as a variable (either as a string or a list of strings), and then only once you are ready to write to the file do you open the file in a `with` block. This means the file is only open for the shortest time necessary, minimising the risk of anything funny happening (such as two applications trying to write at the same time).

In [7]:
# A function which reverses the lines of a file
def reverselines(input_file, output_file):
    with open(input_file, "r") as file:
        lines = file.readlines()
        
    lines.reverse()
    # add a line break to the new first line 
    lines[0] = lines[0] + "\n"
    # remove the line break from the new final line
    lines[-1] = lines[-1][:-1]
    
    with open(output_file, "w") as file:
        file.writelines(lines)
    
    
reverselines("./resources/text_file.txt", "text_reversed.txt")
# you can open the new file via the File menu to check its contents,
# or write some code below which reads and prints it

## Exercise – Pickling
Now that you have learned about two concepts, modules and files, let's see if you can combine them to solve a new problem.

It's very useful to be able to write to and read from a file. Suppose you are writing a game which stores all the previous scores from its players in a dictionary. You probably want the scores to still be there when you restart the game, and the way to do this is write it to the disk, then load it back in the next time the game starts.

In an earlier section we saw an example of a dictionary like this:

In [8]:
recent_scores = {"ray": [5000], "ali": [3000, 7000], "sam": [2000, 1000]}

This works fine, but how do we write it to a text file? We can only write strings to text files.

In [9]:
with open("scores.txt", "w") as file:
    file.write(recent_scores)

TypeError: write() argument must be str, not dict

You will get a similar `TypeError` if you try to use binary mode (`"wb"`) when you open the file.

You could invent your own text-based structure, and convert the dictionary to and from this structure whenever you write it to the disk. But this is error-prone. Encoding and parsing data is difficult to do robustly.

Luckily, there is an inbuilt module called `pickle` which can be used to *serialise* arbitrary Python objects, including dictionaries. 

[Here is the documentation](https://docs.python.org/3/library/pickle.html) for the `pickle` module, see if you can work out how to use this module to save the dictionary to a file. Then in a separate cell, read the file into a *new* dictionary variable (use a different name just to be sure), and find the player with the highest score.

In [ ]:
# save the dictionary to a file

In [ ]:
# load the dictionary into a new variable name

## What Next?

When you are done with this notebook, go back to Engage and move onto the next section.